In [12]:
import numpy as np
import pandas as pd
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb
from keras.utils import pad_sequences

In [2]:
max_features = 20000
maxlen = 80

In [3]:
print('Загрузка данных...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'тренировочные последовательности')
print(len(x_test), 'тестовые последовательности')

Загрузка данных...
25000 тренировочные последовательности
25000 тестовые последовательности


In [4]:
print('Pad последовательности (примеров в x единицу времени)')
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Pad последовательности (примеров в x единицу времени)
x_train shape: (25000, 80)
x_test shape: (25000, 80)


In [5]:
print('Построение модели...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

Построение модели...


In [6]:
def f(batch_size=50, optimizer='adam', epochs=1):
    
    model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])
    
    print('Процесс обучения...')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test))
    score, acc = model.evaluate(x_test, y_test,
                                batch_size=batch_size)
    print('Результат при тестировании:', score)
    print('Тестовая точность:', acc)
    
    return acc

In [7]:
accs = []

for batch_size in [50, 100]:
    for optimizer in ['adam', 'rmsprop']:
        for epochs in [1, 2]:
            accs.append(f(batch_size, optimizer, epochs))

Процесс обучения...
500/500 [==============================] - 16s 32ms/step - loss: 0.3524 - accuracy: 0.8448
Результат при тестировании: 0.3524385094642639
Тестовая точность: 0.844760000705719
Процесс обучения...
Epoch 1/2
500/500 [==============================] - 142s 278ms/step - loss: 0.2535 - accuracy: 0.8968 - val_loss: 0.3780 - val_accuracy: 0.8310
Epoch 2/2
500/500 [==============================] - 16s 33ms/step - loss: 0.4348 - accuracy: 0.8297
Результат при тестировании: 0.4348355531692505
Тестовая точность: 0.8296800255775452
Процесс обучения...
500/500 [==============================] - 15s 29ms/step - loss: 0.5568 - accuracy: 0.8295
Результат при тестировании: 0.5567550659179688
Тестовая точность: 0.829479992389679
Процесс обучения...
Epoch 1/2
500/500 [==============================] - 119s 234ms/step - loss: 0.0849 - accuracy: 0.9709 - val_loss: 0.5562 - val_accuracy: 0.8293
Epoch 2/2
500/500 [==============================] - 15s 30ms/step - loss: 0.8170 - accuracy: 

In [9]:
pd.DataFrame({'batch_size': [50, 50, 50, 50],
             'optimizer': ['adam', 'adam', 'rmsprop', 'rmsprop'],
             'epochs': [1, 2, 1, 2],
             'accuracy': [0.83732, 0.82568, 0.83568, 0.84696]})

,batch_size,optimizer,epochs,accuracy
0,50,adam,1,0.83732
1,50,adam,2,0.82568
2,50,rmsprop,1,0.83568


In [10]:
pd.DataFrame({'accuracy': accs},
            index=pd.MultiIndex.from_product([[50, 100], ['adam', 'rmsprop'], [1, 2]], 
                                             names=['batch_size', 'optimizer', 'epochs']))

accuracy
batch_size optimizer epochs          
50         adam      1        0.84476
                     2        0.82968
           rmsprop   1        0.82948
                     2        0.80940
100        adam      1        0.81956
                     2        0.80936
           rmsprop   1        0.81348
                     2        0.81520